In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-2.02658659e-04  8.14802200e-02  3.13617885e-02  2.92059244e-03
   2.61564925e-02  2.90740076e-02  7.82618374e-02 -1.80420070e-03
   1.01344310e-01 -4.51711901e-02  5.84350154e-02 -1.53201260e-02
   5.49956188e-02 -9.86435339e-02 -3.50253284e-02  8.45673773e-03
   1.58607960e-02  1.05626667e-02 -3.42709795e-02 -4.75063315e-03
   9.99022052e-02 -2.06018202e-02 -4.47837450e-02  3.12135834e-02
  -1.19240591e-02 -5.15016057e-02 -1.33605814e-02  1.89621951e-02
   9.76810157e-02 -5.44110313e-02 -3.43313850e-02  8.12905505e-02
   4.88119796e-02 -1.10284034e-02  2.13518236e-02  1.27189439e-02
  -1.43967969e-02  3.62864174e-02 -7.61233270e-02  3.23293880e-02
   2.08102670e-02 -4.22015898e-02  9.12907422e-02  2.08530016e-02
  -3.08017135e-02 -8.38505328e-02  1.30890692e-02 -3.00630834e-02
   4.11229767e-02 -1.27495319e-01 -7.78027102e-02 -3.93412188e-02
   1.52594782e-03 -2.80108862e-02  3.41662765e-02  1.46712540e-02
  -7.71653503e-02  1.63619563e-01  4.11295518e-02 -5.24460115e-02
  -4.18772

In [3]:
! git clone https://github.com/cyberytti/ToolHunt

Cloning into 'ToolHunt'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 146.15 KiB | 6.09 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
!pip install pyngrok
# =========================
# Directly Run ToolHunt in Colab (Debugging Mode)
# =========================

import os
import sys
import threading
import time
from pyngrok import ngrok

# --- 1. Set up paths and environment ---
project_root = "/content" # Standard Colab working directory
toolhunt_dir = os.path.join(project_root, "ToolHunt")

if not os.path.isdir(toolhunt_dir):
    raise FileNotFoundError(f"ToolHunt directory not found at {toolhunt_dir}")

# Crucially, add the project root to Python's path
# This allows 'from backend.main import ...' to work when app.py is run
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

print(f"Project root: {project_root}")
print(f"ToolHunt dir: {toolhunt_dir}")
print(f"sys.path (relevant parts): {[p for p in sys.path if 'content' in p]}")

# Set ngrok token
ngrok.set_auth_token('Your ngrok authentication key')

# --- 2. Import and run Flask app ---
try:
    # Change working directory to ToolHunt so relative paths inside app.py work
    # (like loading templates)
    os.chdir(toolhunt_dir)
    print(f"Changed working directory to: {os.getcwd()}")

    # Now we can import app.py as a module because we added /content to sys.path
    # and we are running from within the ToolHunt directory for relative file paths.
    import app

    print("✅ Successfully imported app.py")

    # --- 3. Run Flask in a separate thread ---
    def run_flask():
        print("🚀 Starting Flask app...")
        # Run Flask app. Note: debug=True might cause issues in threads/Colab,
        # but let's try it first. If problems occur, remove debug=True.
        app.app.run(host='127.0.0.1', port=5000, debug=False) # Start without debug first

    flask_thread = threading.Thread(target=run_flask)
    flask_thread.daemon = True # Dies when main thread dies
    flask_thread.start()
    print("🧵 Flask thread started.")

    # --- 4. Wait and check ---
    # Give Flask time to start (especially for model download on first run)
    print("⏳ Waiting for Flask to initialize...")
    time.sleep(8) # Adjust if needed

    # A simple check if the thread is still alive (not a guarantee the server is up)
    if flask_thread.is_alive():
        print("✅ Flask thread is running.")
    else:
        print("⚠️ Flask thread has stopped. Check for errors in the logs above.")

    # --- 5. Create ngrok tunnel ---
    print("🚇 Starting ngrok tunnel...")
    tunnel = ngrok.connect(5000, "http")
    print("\n🎉 ToolHunt is ready!")
    print(f"🔗 Public URL: {tunnel.public_url}")
    print("\nPress the 'Interrupt Execution' button (⏹️) in Colab to stop.")

    # --- 6. Keep alive ---
    # Keep the main thread alive to maintain the tunnel
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n🛑 Interrupted by user.")
    finally:
        print("🧹 Cleaning up...")
        ngrok.kill()
        # Note: Stopping the Flask thread cleanly from here is tricky in Colab.
        # The daemon thread will stop when the main program ends.
        print("✅ Done.")

except Exception as e:
    print(f"❌ An error occurred during startup: {e}")
    import traceback
    traceback.print_exc()


KeyboardInterrupt: 